# [**Instructions for this workshop**](https://msu-ai.notion.site/Instructions-and-Help-5ba458816042451d8b2fe22597f62361) 

🙏 This workshop is a heavy modification of [work originally done by TechVidvan](https://techvidvan.com/tutorials/python-sentiment-analysis/). Many thanks!

# [Step 1](https://msu-ai.notion.site/Workshop-Instructions-5ba458816042451d8b2fe22597f62361#a71c5a5ce5ea4fa1b0d83d77f865ea52): Download the dataset

Before we can do any data analysis, we need to get the data. Running the following command will download the dataset [from Google Drive](https://drive.google.com/file/d/1Qvbx3wp-8bWMw35MA3nQtke0qYk6EZfg/view?usp=sharing):

In [ ]:
!gdown https://drive.google.com/uc?id=1Qvbx3wp-8bWMw35MA3nQtke0qYk6EZfg

If the command<sup>([*](https://msu-ai.notion.site/Workshop-Instructions-5ba458816042451d8b2fe22597f62361#0a602163f3d44f9e89de12f93f04db12))</sup> is successful, then you don't need to do anything else. The data is now available for your code to use.

If you're feeling paranoid, or just want to manually confirm that the data has been downloaded, you can use the "files" tab on the left. After running the command, you should have a file called `Tweets.csv`. <sup>([*](https://msu-ai.notion.site/Workshop-Instructions-5ba458816042451d8b2fe22597f62361#698a38a07af24d59b33b74aa43d51776))</sup>

# [Step 2](https://msu-ai.notion.site/Workshop-Instructions-5ba458816042451d8b2fe22597f62361#e89e688e952a4b44adfc85eb5eb32a00): Prepare the dataset for use

Let's take a look at the data we just downloaded. The following code will load up the data<sup>([*](https://msu-ai.notion.site/Workshop-Instructions-5ba458816042451d8b2fe22597f62361#75c1fc99ff3745f393368963e6d6d9bd))</sup> from the file, and then display the first 3 rows so we can take a peek.

Go ahead and run it now, and then look at the results.

In [ ]:
import pandas as pd

# Load the tweets as a dataframe
tweets = pd.read_csv("./Tweets.csv")

# Print out the first few rows to make sure it worked
tweets.head(3) # three rows

As you can see, there's a lot of data here. Each row has a whole bunch of information. This is more than we need, so let's simplify it a bit.

We're going to create a copy of the table that only has the columns we care about most. Follow the instructions in the code below to get a table with only the `text` and `airline_sentiment` columns. If you get stuck, please ask for help!

In [ ]:
# This line creates a new dataframe that only includes some of the original columns.
# Can you change it to only include two columns: the tweet text and the airline sentiment?
tweets_simple = tweets[['tweet_id', 'airline', 'name', 'tweet_created']]

# Print out the first few rows of the simplified table
tweets_simple.head(5) # five rows

Check out the `airline_sentiment` column. It tells us whether each tweet is positive or negative. But there are also some neutral tweets. Let's see how many there are of each (positive, negative, and neutral):

In [ ]:
# Checks how many tweets there are with each sentiment:
tweets_simple['airline_sentiment'].value_counts()

Hopefully the above code worked. (If not, it probably means that your simplified tweets table isn't quite correct.)

It looks like the tweets are mostly negative (shocker 🙄), with some neutral and positive tweets mixed in as well.

When it comes to teaching the computer, the neutral tweets are not very helpful. Our goal is to teach the computer the difference between a positive and a negative tweet. Neutral tweets don't really give any new information. So, to simplify our dataset, let's get rid of the neutral tweets.<sup>([*](https://msu-ai.notion.site/Workshop-Instructions-5ba458816042451d8b2fe22597f62361#ec4352550c8d4c8c83504c5c4513ebb1))</sup>

In [ ]:
# Right now we're filtering the dataframe to grab *only* the neutral tweets.
# This is exactly the OPPOSITE of what we want! Can you change it to grab
# everything *except* the neutral tweets?
tweets_filtered = tweets_simple[tweets_simple['airline_sentiment'] == 'neutral']

# Print the first few rows of the new table. How many neutrals do you see?
tweets_filtered.head(10) # ten rows

Computers really like to work with numbers, so when we train it to categorize tweets, it's going to want our categories to just be numbers (category 0, category 1, etc...) rather than labels like "positive" or "negative".

We are going to use the `.factorize()` method to search through the `airline_sentiment` column and give each possible output (in this case, "positive" or "negative") its own number index.

In [ ]:
sentiment_label = tweets_filtered['airline_sentiment'].factorize()
print(sentiment_label)

In the printed result above, the "Index" shows us the order in which the `.factorize()` method chose to put our labels. It started with "positive" (0) and then "negative" (1). (If there were more possibilities, they would be assigned 2, 3, 4, etc...)

So 0 means positive and 1 means negative. And at the beginning of the previous output there is an array of 1s and 0s. This is the column of positives and negatives from our original dataframe converted into number form.

Now that our data is prepared, we're going to create a dictionary of all the words that are used in all of the tweets in our dataset. (Remember, we are teaching the computer to read completely from scratch, so this dictionary is going to come in handy.)

To begin, let's grab the tweet text column and turn it into a plain python list of strings. You don't need to change anything here. Just run the code:

In [ ]:
tweet_strings = tweets_filtered["text"].values # Get just the values in the text column

print(tweet_strings) # Print out the tweets (just a few examples)
print(len(tweet_strings)) # Print the total number of tweets

Now that we have the tweets in text form, we can create a dictionary of words.<sup>([*](https://msu-ai.notion.site/Workshop-Instructions-5ba458816042451d8b2fe22597f62361#540adbe36d004763b81cefad576efb51))</sup> This dictionary is going to assign a unique number to each word, starting with 1 and counting up. The 1st word will be the most common word used in the tweets, and the list will be sorted in decreasing usage from there.

Let's give it a whirl:

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Create a dictionary with no more than 5000 words
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(tweet_strings) # ...using our list of tweets

# Print out the dictionary
print(tokenizer.word_index)

As you can see, the above code successfuly produced a dictionary! Each word now has a number assigned to it.

We can now use this dictionary to convert any sentence into number form. We'll try it out on the sentence "I got delayed":

In [ ]:
tokenizer.texts_to_sequences(["I got delayed"])

Do these numbers look right? They should! Check each number in the dictionary and make sure it corresponds to the correct word. Then, try typing in your own sentence, and see if you can predict which numbers it will correspond to:

In [ ]:
text = input("Enter your sentence to convert to numbers: ")
tokenizer.texts_to_sequences([text])

**Bonus question:** What happens if you enter a sentence that uses a word which is not in the dictionary?

**Bonus question:** If you look through the dictionary, can you find any words that seem like they would be unhelpful when trying to determine sentiment? A big part of machine learning is cleaning up data, and we haven't been super careful about that here.

---

Now that we have our dictionary, we're going to convert *ALL* the tweets in the entire dataset to number form. Then we'll print out the first three tweets in number form, just to make sure it worked:

In [ ]:
# Encode ALL the tweets as numbers
encoded_docs = tokenizer.texts_to_sequences(tweet_strings)

print(len(encoded_docs)) # Print the number of tweets that got encoded

encoded_docs[:3] # Print just the first 3 encoded tweets

Amazing!

One problem we have is that these tweets all have different numbers of words, but our machine learning model is going to expect each tweet to be equal in length. That's admittedly a bit of an unusual requirement, but it's just how machine learning goes. To solve this problem, we will put a bunch of zeroes at the front of each tweet so that they are all 200 words (numbers) long.

Currently, this code trims each tweet to be exactly 4 words long. Can you fix it?

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Pad tweets so that they are all the same length.
# We SHOULD be padding these to a length of 200, but currently the code is wrong.
padded_sequence = pad_sequences(encoded_docs, maxlen=4)

print(padded_sequence)

You should see a summary above with a block of numbers, where each tweet starts with a bunch of zeroes and ends with actual numbers corresponding to words.

Now we're all done with this process! Just to summarize, let's take a look at how we processed one particular tweet. (This code isn't going to do anything new, just show us what we've already achieved.)

In [ ]:
# First, we had the tweet in text form:
print('As text:', tweet_strings[0])

# Then we converted it to numbers using our dictionary:
print('As numbers:', encoded_docs[0])

# Finally, we added zeroes to make the tweet exactly 200 words long:
print('As exactly 200 numbers:', padded_sequence[0])

(If you want to take a look at the same information for any other tweet, you can change the code above to use an index other than 0.)

# [Step 3](https://msu-ai.notion.site/Workshop-Instructions-5ba458816042451d8b2fe22597f62361#ec7a956c644a46edae090704ce3d76d2): Design the model

Finally! We have our tweets and sentiments encoded as numbers. Now we can get to the fun stuff.

The following code defines the design of our machine learning model. Hopefully this model will be good at categorizing tweets. For now, just run the code and don't worry too much about exactly how it works. We have some resources explaining how the model works, but the step after this one is very time consuming (we have to wait for the computer to do a bunch of math), so we'll read about it then.

Okay... run this code! It should print a summary of the model we've designed.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, SpatialDropout1D

# We design our machine learning model here. (Learn more in the instructions document.)
model = Sequential([
    # Convert word ids to vectors representing the meaning of the word (http://projector.tensorflow.org/)
    Embedding(len(tokenizer.word_index), 10, input_length=200),

    # Long short-term memory. This allows the model to read one word at a team, and "remember" some information about what it has already read.
    LSTM(30),

    # Take the information from the previous layer and condense it into a single output
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

# [Step 4](https://msu-ai.notion.site/Workshop-Instructions-5ba458816042451d8b2fe22597f62361#db3950c639424a2ea1be0ba557d82ed8): Train the model

Awesome! We have our model designed.

Now, go ahead and run the code below. Its job is to take the model we designed in step 3 and train it based on our data which we prepared in step 2. You'll notice that there is a progress bar, and it's not super quick.

**While you wait, read the ["Understanding the model"](https://msu-ai.notion.site/Workshop-Instructions-5ba458816042451d8b2fe22597f62361#fffa31b9682946a09a1817eff4dd73f0) section of the instructions.**

In [ ]:
history = model.fit(
    padded_sequence,
    sentiment_label[0],
    validation_split=0.2,
    epochs=2,
    batch_size=32
)

Hopefully as the model trained, its accuracy improved.<sup>([*](https://msu-ai.notion.site/Workshop-Instructions-5ba458816042451d8b2fe22597f62361#7d455f4d7f5c45469ff60808f28d9615))</sup> The accuracy is the percentage of tweets from the dataset that the model can predict accurately. To learn more about accuracy vs. validation accuracy, read [the answer in the instructions document](https://msu-ai.notion.site/Workshop-Instructions-5ba458816042451d8b2fe22597f62361#01da8130b7c94776be70ac1e71dfd82c).

The following code prints out a graph<sup>([*](https://msu-ai.notion.site/Workshop-Instructions-5ba458816042451d8b2fe22597f62361#62ddec9be50e4d0fa27acb612910c742))</sup> of the accuracy and validation accuracy over the course of training:

In [ ]:
import matplotlib.pyplot as plt

# Graph the accuracy and validation accuracy
plt.plot(history.history['accuracy'], label='Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation accuracy')

# Graph the loss and validation loss
# plt.plot(history.history['loss'], label='Loss')
# plt.plot(history.history['val_loss'], label='Validation loss')

plt.legend()
plt.show()

# [Step 5](https://msu-ai.notion.site/Workshop-Instructions-5ba458816042451d8b2fe22597f62361#54b84843c5904b0ba7dab1b7144d36ec): Use the model

Now that our model is designed and trained, we can actually use it to make predictions! If all has gone well, our model should be able to look at a tweet it has never seen before and successfully classify it as either positive or negative.

Let's try one positive and one negative tweet and see if it works:


In [ ]:
def predict_sentiment(text):
    tw = tokenizer.texts_to_sequences([text])
    tw = pad_sequences(tw,maxlen=200)
    prediction = int(model.predict(tw).round().item())
    return sentiment_label[1][prediction]

test_sentence1 = "I enjoyed my journey on this flight."
print("Tweet text: ", test_sentence1)
print("Predicted label: ", predict_sentiment(test_sentence1))
print()

test_sentence2 = "This was the worst flight experience of my life!"
print("Tweet text: ", test_sentence2)
print("Predicted label: ", predict_sentiment(test_sentence2))

Amazing! And it's not just these tweets. In theory, it should work for anything you type in! Give it a try:

In [ ]:
text = input("Enter your tweet: ")
print("Predicted label: ", predict_sentiment(text))

**Bonus question:** Can you find a tweet that tricks the model into predicting incorrectly?

# ⭐ [Bonus Step](https://msu-ai.notion.site/Workshop-Instructions-5ba458816042451d8b2fe22597f62361#5b15b0e0f47b42738492686a9bb1dc7a): Explore & improve the model

At this point, we have a working model! We could stop here, but it's far more interesting to continue exploring the data, asking questions, and improving the model.

### Questions to consider:
- **Is it possible to create a simpler model that is still effective?** In the model summary, it lists the total number of parameters (knobs and dials) that need to be learned during training. How few can you use? Is there a sweet spot for complexity?
- **Can you change the model to be more effective?** The best measure of the model's effectiveness is validation accuracy. How great can you make it?
- **Based on our dataset, which airlines receive the most positive or the most negative feedback?** Who should I fly with? (You’ll have to return to step 2, where we cleaned up the dataset, and change the process so that you don’t remove some of the columns that were originally there.)
- **Can you create a version of the model that uses *only* Dense layers?** How effective is this version? Why do you think it is better/worse?
    - What do you notice about the training time for this model? Why do you think it takes longer/shorter to train?
- **What happens if you change the number of words in the dictionary created by the tokenizer?** Can you still create an effective model using a far smaller dictionary?
- **Can you design a model that intentionally overfits the data?** This means that its `accuracy` during training is much higher is much higher than its validation accuracy (`val_accuracy`). (Overfitting makes the model worse, but it's an interesting experiment to perform.)